In [ ]:
# Todo

0. Improve model to 50% accuracy
1. Data Preprocessing (3 layer structure) [DONE]
2. Data Preprocessing (5 layer structure)
3. Building agent for the deep learning [Consider valid move]
4. Reading the reinforcement learning part

In [ ]:
import tensorflow as tf
from sklearn.utils import shuffle
import numpy as np
from tqdm.notebook import tqdm
from numba import jit
import os
import aim
from aim.tensorflow import AimCallback 
from aim import Session
from encoder.base import get_encoder_by_name
encoder = get_encoder_by_name('layer_20_encoder', 8)

In [ ]:
# Loading data

In [ ]:
mat_files_X = sorted(["game_data/" + file for file in os.listdir("game_data/") if file.startswith("master_mat_rule")])
raw_X = np.concatenate([np.load(file) for file in mat_files_X])
raw_X.shape

In [ ]:
mat_files_Y = sorted(["game_data/" + file for file in os.listdir("game_data/") if file.startswith("master_move_rule")])
raw_Y = np.concatenate([np.load(file) for file in mat_files_Y])
raw_Y.shape

In [ ]:
mat = raw_X[10]

In [ ]:
zeros = np.where(mat == 0)
unvisited_node_list = list(zip(zeros[0], zeros[1]))

In [ ]:
mat

In [ ]:
@jit()
def break_down_layers(game_state):
    mat = np.copy(game_state)

    master_X_open_self = []
    master_X_open_oppo = []

    for search_num in np.arange(4,0,-1):
        X_open_self = np.zeros((8,8))
        X_open_oppo = np.zeros((8,8))

        # Did not consider open and close siutation
        m = 8
        n = 8
        for i in range(m):
            for j in range(m):
                if j + search_num+1 <= m:
                    sideway = mat[i][j:j+(search_num)]
                    if np.sum(sideway) ==search_num:
                        X_open_self[i][j:j+search_num] = 1
                    if np.sum(sideway) == -search_num:
                        X_open_oppo[i][j:j+search_num] = 1

                if i + search_num+1 <= m:
                    vert = mat[:,j][i:i+(search_num)]
                    if np.sum(vert) == search_num:
                        X_open_self[:,j][i:i+search_num] = 1
                    if np.sum(vert) == -search_num:
                        X_open_oppo[:,j][i:i+search_num] = 1

                if j + search_num+1 <= m and i + search_num+1 <= n:
                    diag = [mat[i+x][j+y] for x in range(search_num) for y in range(search_num) if x == y]
                    if np.sum(diag) == search_num:
                        for k in range(search_num):
                            X_open_self[i+k][j+k] = 1
                    if np.sum(diag) == -search_num:
                        for k in range(search_num):
                            X_open_oppo[i+k][j+k] = 1

                if j - search_num >= 0 and i + search_num+1 <= n:
                    diag = [mat[i+x][j-y] for x in range(search_num) for y in range(search_num) if x == y]
                    if np.sum(diag) == search_num:
                        for k in range(search_num):
                            X_open_self[i+k][j-k] = 1
                    if np.sum(diag) == -search_num:
                        for k in range(search_num):
                            X_open_oppo[i+k][j-k] = 1

        i_axis = list(np.where(X_open_self==1)[0]) + list(np.where(X_open_oppo==1)[0])
        j_axis = list(np.where(X_open_self==1)[1]) + list(np.where(X_open_oppo==1)[1])

        for i,j in zip(i_axis, j_axis):
            mat[i][j] = 0

        master_X_open_self.append(np.copy(X_open_self))
        master_X_open_oppo.append(np.copy(X_open_oppo))
    return master_X_open_self + master_X_open_oppo

In [ ]:
# No numba time: 13:52
@jit()
def potential_open_moves(game_state):

    mat = np.copy(game_state)

    master_X_open_self = []
    master_X_open_oppo = []

    for search_num in np.arange(4,1,-1):
        X_open_self = np.zeros((8,8))
        X_open_oppo = np.zeros((8,8))
        X_record_self = np.zeros((8,8))
        X_record_oppo = np.zeros((8,8))

        # Did not consider open and close siutation
        m = 8
        n=8
        for i in range(m):
            for j in range(m):

                if j + search_num+1 <= m:
                    sideway = mat[i][j:j+(search_num)]
                    if np.sum(sideway) ==search_num:
                        X_record_self[i][j:j+search_num] = 1

                        if mat[i][j-1] == 0:
                            X_open_self[i][j-1] = 1
                        if  mat[i][j+(search_num)] == 0:
                            X_open_self[i][j+(search_num)] = 1

                    if np.sum(sideway) == -search_num:
                        X_record_oppo[i][j:j+search_num] = 1

                        if mat[i][j-1] == 0:
                            X_open_oppo[i][j-1] = 1
                        if  mat[i][j+(search_num)] == 0:
                            X_open_oppo[i][j+(search_num)] = 1

                if i + search_num+1 <= m:
                    vert = mat[:,j][i:i+(search_num)]
                    if np.sum(vert) == search_num:
                        X_record_self[:,j][i:i+search_num] = 1

                        if mat[i-1][j] == 0:
                            X_open_self[i-1][j] = 1
                        if mat[i+(search_num)][j] == 0:
                            X_open_self[i+(search_num)][j] = 1

                    if np.sum(vert) == -search_num:
                        X_record_oppo[:,j][i:i+search_num] = 1

                        if mat[i-1][j] == 0:
                            X_open_oppo[i-1][j] = 1
                        if mat[i+(search_num)][j] == 0:
                            X_open_oppo[i+(search_num)][j] = 1


                if j + search_num+1 <= m and i + search_num+1 <= n:
                    diag = np.array([mat[i+x][j+y] for x in range(search_num) for y in range(search_num) if x == y])
                    if np.sum(diag) == search_num:
                        for k in range(search_num):
                            X_record_self[i+k][j+k] = 1


                        if mat[i-1][j-1] == 0:
                            X_open_self[i-1][j-1] = 1
                        if mat[i+(search_num)][j+(search_num)] == 0:
                            X_open_self[i+(search_num)][j+(search_num)] = 1

                    if np.sum(diag) == -search_num:
                        for k in range(search_num):
                            X_record_oppo[i+k][j+k] = 1

                        if mat[i-1][j-1] == 0:
                            X_open_oppo[i-1][j-1] = 1
                        if mat[i+(search_num)][j+(search_num)] == 0:
                            X_open_oppo[i+(search_num)][j+(search_num)] = 1

                if j - search_num >= 0 and i + search_num+1 <= n:
                    diag = np.array([mat[i+x][j-y] for x in range(search_num) for y in range(search_num) if x == y])
                    if np.sum(diag) == search_num:
                        for k in range(search_num):
                            X_record_self[i+k][j-k] = 1

                        if mat[i-1][j+1] == 0:
                            X_open_self[i-1][j-1] = 1
                        if  mat[i+(search_num)][j-(search_num)] == 0:
                            X_open_self[i+(search_num)][j-(search_num)] = 1
                    if np.sum(diag) == -search_num:
                        for k in range(search_num):
                            X_record_oppo[i+k][j-k] = 1


                        if mat[i-1][j+1] == 0:
                            X_open_oppo[i-1][j+1] = 1
                        if mat[i+(search_num)][j-(search_num)] == 0:
                            X_open_oppo[i+(search_num)][j-(search_num)] = 1

        i_axis = list(np.where(X_record_self==1)[0]) + list(np.where(X_record_oppo==1)[0])
        j_axis = list(np.where(X_record_self==1)[1]) + list(np.where(X_record_oppo==1)[1])

        for i,j in zip(i_axis, j_axis):
            mat[i][j] = 0

        master_X_open_self.append(np.copy(X_open_self))
        master_X_open_oppo.append(np.copy(X_open_oppo))
    return master_X_open_self + master_X_open_oppo

In [ ]:
def preprocess_data(raw_X, raw_Y):
    total_X = []
    for i in tqdm(range(1, len(raw_X))):
        input_X = raw_X[i]
        layer_my_move = np.array(input_X > 0).astype(int)
        layer_oppo_move = np.array(input_X < 0).astype(int)
        layer_valid_move = np.array(input_X == 0).astype(int)
        #layers_break_down = break_down_layers(input_X)
        layers_open_moves = potential_open_moves(input_X)
        layer_last_move = raw_Y[i-1]
        layer_zeros = np.zeros((8,8))
        layer_ones = np.ones((8,8))
        final_X = np.dstack([layer_my_move, layer_oppo_move, layer_last_move, layer_valid_move+layer_zeros+layer_ones] + layers_open_moves)
        final_X = np.expand_dims(final_X, axis = 0)
        total_X.append(final_X)
    X = np.vstack(total_X)
    Y = raw_Y[1:]
    return X,Y

In [ ]:
X,Y = preprocess_data(raw_X, raw_Y)


In [ ]:
X,Y = shuffle(X,Y)
X_train = X[:int(len(X)*0.8)]
X_test = X[int(len(X)*0.8):]
Y_train = Y[:int(len(X)*0.8)]
Y_test = Y[int(len(X)*0.8):]
print(X_train.shape)
Y_train = Y_train.reshape(Y_train.shape[0], 64)
Y_test = Y_test.reshape(Y_test.shape[0],64)
print(X_train.shape)

In [ ]:
# Data Generator

In [ ]:
class DataGenerator:
    def __init__(self, data_directory, samples):
        self.data_directory = data_directory
        self.samples = samples
        self.files = set(file_name for file_name, index in samples)    1
        self.num_samples = None

    def get_num_samples(self, batch_size=128, num_classes=19 * 19):    2
        if self.num_samples is not None:
            return self.num_samples
        else:
            self.num_samples = 0
            for X, y in self._generate(batch_size=batch_size,
                                       num_classes=num_classes):
                self.num_samples += X.shape[0]
            return self.num_samples
        
    def _generate(self, batch_size, num_classes):
        for zip_file_name in self.files:
            file_name = zip_file_name.replace('.tar.gz', '') + 'train'
            base = self.data_directory + '/' + file_name + '_features_*.npy'
            for feature_file in glob.glob(base):
                label_file = feature_file.replace('features', 'labels')
                x = np.load(feature_file)
                y = np.load(label_file)
                x = x.astype('float32')
                y = to_categorical(y.astype(int), num_classes)
                while x.shape[0] >= batch_size:
                    x_batch, x = x[:batch_size], x[batch_size:]
                    y_batch, y = y[:batch_size], y[batch_size:]
                    yield x_batch, y_batch    
                    
    def generate(self, batch_size=128, num_classes=19 * 19):
        while True:
            for item in self._generate(batch_size, num_classes):
                yield item

In [ ]:
# Transforming data shape

In [ ]:
#X_train = X_train.reshape(X_train.shape[0],8,8,1)
Y_train = Y_train.reshape(Y_train.shape[0], 64)
#X_test = X_test.reshape(X_test.shape[0],8,8,1)
Y_test = Y_test.reshape(Y_test.shape[0],64)
print(X_train.shape)

In [ ]:
class GomokuNet(tf.keras.Model):
    def __init__(self,nums_class=64):
        super(GomokuNet,self).__init__()
        self.model = tf.keras.layers.Conv2D(48,(3,3), input_shape=(8,8,10),strides=(1,1))
        self.res_layer_1 = self.ResNet_build(48, 2, strides=1)
        self.avg_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc_model = tf.keras.layers.Dense(nums_class)
    
    def call(self, inputs, training=None):
        x = self.model(inputs)
        x = self.res_layer_1(x)
        x = self.avg_pool(x) 
        x = self.fc_model(x)
        return x
    
    def res_net_block(input_data, filters, conv_size):
        x = tf.keras.layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([x, input_data])
        x = tf.keras.layers.Activation('relu')(x)
        return x
    
    def ResNet_build(self,filter_nums,block_nums,strides=1):
        build_model = tf.keras.models.Sequential()
        build_model.add(ResBlock(filter_nums,strides))
        for _ in range(1,block_nums):
            build_model.add(ResBlock(filter_nums,strides=1))
        return build_model
    

class ResBlock(tf.keras.layers.Layer):
    def __init__(self, filter_nums, strides=1, residual_path=False):
        super(ResBlock, self).__init__()

        self.conv_1 = tf.keras.layers.Conv2D(filter_nums,(3,3),strides=strides,padding='same')
        self.bn_1 = tf.keras.layers.BatchNormalization()
        self.act_relu = tf.keras.layers.Activation('relu')

        self.conv_2 = tf.keras.layers.Conv2D(filter_nums,(3,3),strides=1,padding='same')
        self.bn_2 = tf.keras.layers.BatchNormalization()

        if strides !=1:
            self.block = tf.keras.models.Sequential()
            self.block.add(tf.keras.layers.Conv2D(filter_nums,(1,1),strides=strides))
        else:
            self.block = lambda x:x


    def call(self, inputs, training=None):

        x = self.conv_1(inputs)
        x = self.bn_1(x, training=training)
        x = self.act_relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x,training=training)

        identity = self.block(inputs)
        outputs = tf.keras.layers.add([x,identity])
        outputs = tf.nn.relu(outputs)

        return outputs

In [ ]:
aim_session = Session(experiment='Gomuko-4planes-6layers-normalshape')

In [ ]:
model = GomokuNet()

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (4,4), activation='relu', input_shape=(8,8,10), padding = 'same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (2,2), activation='relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (4,4), activation='relu',  padding = 'same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', ddpadding = 'same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding = 'same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    #The third convolution
    tf.keras.layers.Conv2D(64, (2,2), activation='relu', padding = 'same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (2,2), activation='relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    
    
    tf.keras.layers.Flatten(),
    
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(256, activation='relu'),
    
    # Last layer of model
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='softmax')])

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['accuracy'])

history = model.fit(X_train,
                    Y_train, 
                    epochs=30,
                    batch_size=256,
                    validation_data = (X_test, Y_test), 
                    verbose = 1, 
                   #callbacks=[AimCallback(aim_session)]
                   )

In [ ]:
# Training RL model

In [ ]:
import h5py
import tempfile
import keras_util
import numpy as np
import tensorflow as tf
from encoder.base import get_encoder_by_name
encoder = get_encoder_by_name('allpattern', 8)
from tensorflow.keras.models import load_model, save_model

In [ ]:
with h5py.File("experiment_2020_12_22_06_29_50", 'r+') as f:
    state = np.array(f['experience']['states'])
    action = np.array(f['experience']['actions'])
    reward = np.array(f['experience']['rewards'])    

In [ ]:
reward = np.where(reward == 0.2, -0.1,reward)

In [ ]:
with h5py.File("experience_data/experiment_updated", 'w') as f:
    f.create_group('experience')
    f['experience'].create_dataset('states', data=state_updated)
    f['experience'].create_dataset('actions', data=action_updated)
    f['experience'].create_dataset('rewards', data=reward_updated)

In [ ]:
# Training value network

In [ ]:
with h5py.File("experiment_2020_12_22_06_29_50", 'r+') as f:
    state = np.array(f['experience']['states'])
    reward = np.array(f['experience']['rewards'])   
    
win_lose_index = np.where(reward != 0.2)
state = state[win_lose_index]
reward = reward[win_lose_index]

In [ ]:
from sklearn.utils import shuffle
state,reward = shuffle(state, reward)

In [ ]:
state = np.squeeze(state, axis = 1)

In [ ]:
X_train = state[:int(len(state)*0.8)]
X_test = state[int(len(state)*0.8):]
reward_train = reward[:int(len(state)*0.8)]
reward_test = reward[int(len(state)*0.8):]

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(8,8,20), padding = 'same'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(64, (5,5), activation='relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (4,4), activation='relu', padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv2D(1, (2,2), activation='relu', padding = 'same'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),

    # 512 neuron hidden layer
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Last layer of model
    tf.keras.layers.Dense(1, activation='tanh')
    ])

In [ ]:
Early_Stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

model.compile(loss = 'mse', 
              optimizer='adam')

history = model.fit(X_train,reward_train, 
                    epochs=30,
                    batch_size=256,
                    validation_data = (X_test, reward_test), 
                    verbose = 1, 
                    callbacks=[Early_Stopping_callback]
                   )


In [ ]:
model.save("alpha_gomuku_value_net_no_draw")

# Debug

Where is the problems:
1. No draw hanlding will distort the order (Rejected. Checked both 1 lose to -1 and -1 lose to 1) <br>
2. Shouldnt pre-set starting position <br>
3. Something wrong with -1 (Rejected. Checked the -1 situation for 1 lose to -1 and -1 lose to 1) <br>
4. Wrong setting of learning rate and batch size 
5. Learning rate is too large
6. Batch size is too small

In [ ]:
experience_data/experiment_no_draw_experiment_2020_12_26_07_21_53

In [170]:
with h5py.File(f"experiment_no_draw_experiment_2020_12_26_07_21_53", 'r+') as f:
    raw_state = np.array(f['experience']['raw_states'])
    state = np.array(f['experience']['states'])
    action = np.array(f['experience']['actions'])
    reward = np.array(f['experience']['rewards'])    
    advantage = np.array(f['experience']['advantages']) 

(False, 0)

In [89]:
mat = np.array(
      [[ 1., -1., -1.,  1., -1.,  1., -1.,  1.],
       [-1.,  1., -1.,  1., -1.,  1., -1.,  1.],
       [-1.,  1., -1.,  1.,  1., -1., -1.,  1.],
       [-1., -1.,  1., -1.,  1.,  1.,  1.,  1.],
       [ 1., -1.,  1.,  1., -1.,  1., -1., -1.],
       [ 1.,  1., -1.,  0., -1.,  1.,  1., -1.],
       [ 1., -1., -1., -1.,  1., -1., -1., -1.],
       [ -1.,  1., -1., -1.,  1., -1.,  1.,  1.]]
)

In [90]:
check_for_done(mat)

(False, 0)

In [78]:
for i in range(0, 10000):
    if check_for_done(raw_state[i])[0] is True:
        print(i)

In [ ]:
def check_for_done(mat):
    """
    please write your own code testing if the game is over. Return a boolean variable done. If one of the players wins
    or the tie happens, return True. Otherwise return False. Print a message about the result of the game.
    input:
        2D matrix representing the state of the game
    output:
        none
    """
    m, n = mat.shape
    target1 = [1, 1, 1, 1, 1]
    target2 = [-1, -1, -1, -1, -1]
    if len(np.where(mat == 0)[0]) == 0:
        return True, 0.5

    for i in range(m):
        for j in range(n):
            if j + 5 <= m:
                sideway = mat[i][j:j + 5]
                if (sideway == target1).all():
                    return True, 1
                if (sideway == target2).all():
                    return True, -1
            if i + 5 <= m:
                vert = mat[:, j][i:i + 5]
                if (vert == target1).all():
                    return True, 1
                if (vert == target2).all():
                    return True, -1
            if j + 5 <= m and i + 5 <= n:
                diag = [mat[i + x][j + y] for x in range(5) for y in range(5) if x == y]
                if diag == target1:
                    return True, 1
                if diag == target2:
                    return True, -1
            if j - 4 >= 0 and i + 5 <= n:
                diag = [mat[i + x][j - y] for x in range(5) for y in range(5) if x == y]
                if diag == target1:
                    return True, 1
                if diag == target2:
                    return True, -1
    return False, 0

# Append files

In [219]:
with h5py.File(f"experience_data/experiment_no_draw_experiment_2020_12_26_07_21_53", 'r+') as f:
    raw_state = np.array(f['experience']['raw_states'])
    state = np.array(f['experience']['states'])
    action = np.array(f['experience']['actions'])
    reward = np.array(f['experience']['rewards'])    
    advantage = np.array(f['experience']['advantages']) 

In [220]:
master_raw_state = raw_state
master_state = state
master_action = action
master_reward = reward

In [221]:
files = ["experiment_no_draw_experiment_2020_12_26_07_21_58",
       "experiment_no_draw_experiment_2020_12_26_07_22_25",
       "experiment_no_draw_experiment_2020_12_26_07_22_54"]

In [222]:
for file in files:
    with h5py.File(f"experience_data/{file}", 'r+') as f:
        raw_state = np.array(f['experience']['raw_states'])
        state = np.array(f['experience']['states'])
        action = np.array(f['experience']['actions'])
        reward = np.array(f['experience']['rewards'])    
        advantage = np.array(f['experience']['advantages'])    

    raw_state_updated = raw_state[np.where(reward != 0.0)[0]]
    state_updated = state[np.where(reward != 0.0)[0]]
    action_updated = action[np.where(reward != 0.0)[0]]
    reward_updated = reward[np.where(reward != 0.0)[0]]
    advantage_updated = advantage[np.where(reward != -0.1)[0]]
    
    master_raw_state = np.concatenate([master_raw_state, raw_state_updated], axis =0)
    master_state = np.concatenate([master_state, state_updated], axis =0)
    master_action = np.concatenate([master_action, action_updated], axis =0)
    master_reward = np.concatenate([master_reward, reward_updated], axis =0)

In [225]:
with h5py.File(f"summary_experience_preset_start", 'w') as f:
    f.create_group('experience')
    f['experience'].create_dataset('raw_states', data=master_raw_state)
    f['experience'].create_dataset('states', data=master_state)
    f['experience'].create_dataset('actions', data=master_action)
    f['experience'].create_dataset('rewards', data=master_reward)

# Convert files

In [ ]:
for file in files:
    with h5py.File(f"experience_data/{file}", 'r+') as f:
        raw_state = np.array(f['experience']['raw_states'])
        state = np.array(f['experience']['states'])
        action = np.array(f['experience']['actions'])
        reward = np.array(f['experience']['rewards'])    
        advantage = np.array(f['experience']['advantages'])    

    raw_state_updated = raw_state[np.where(reward != 0.0)[0]]
    state_updated = state[np.where(reward != 0.0)[0]]
    action_updated = action[np.where(reward != 0.0)[0]]
    reward_updated = reward[np.where(reward != 0.0)[0]]
    advantage_update = advantage[np.where(reward != -0.1)[0]]

#     with h5py.File(f"experience_data/experiment_no_draw_{file}", 'w') as f:
#         f.create_group('experience')
#         f['experience'].create_dataset('raw_states', data=raw_state_updated)
#         f['experience'].create_dataset('states', data=state_updated)
#         f['experience'].create_dataset('actions', data=action_updated)
#         f['experience'].create_dataset('rewards', data=reward_updated)
#         f['experience'].create_dataset('advantages', data=advantage_update)